In [138]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.utils.multiclass import unique_labels



In [139]:
data = pd.read_csv("clean.csv") 
datatest = pd.read_csv("test.csv")
data.head()

,keyword,location,text,target
0,NaN,NaN,"['deed', 'reason', 'earthquake', 'may', 'allah...",1
1,NaN,NaN,"['forest', 'fire', 'near', 'la', 'ronge', 'sas...",1
2,NaN,NaN,"['resident', 'ask', 'shelter', 'place', 'notif...",1
3,NaN,NaN,"['people', 'receive', 'wildfire', 'evacuation'...",1
4,NaN,NaN,"['get', 'sent', 'photo', 'ruby', 'alaska', 'sm...",1


In [140]:
# Bag of words representation using CountVectorizer
count_vector = CountVectorizer(ngram_range = (1,2))

X_train = count_vector.fit_transform(data['text'])
X_test = count_vector.transform(datatest['text'])
y_train = data['target']
y_test = datatest['target']

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=72)

KeyError: 'target'

In [128]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [129]:
lr.score(X_test, y_test)

0.8961038961038961

In [130]:
datatest = pd.read_csv("testclean.csv") 
datatest.head()

,id,keyword,location,text
0,0,NaN,NaN,"['happen', 'terrible', 'car', 'crash']"
1,2,NaN,NaN,"['heard', 'earthquake', 'different', 'city', '..."
2,3,NaN,NaN,"['forest', 'fire', 'spot', 'pond', 'geese', 'f..."
3,9,NaN,NaN,"['apocalypse', 'light', 'spokane', 'wildfire']"
4,11,NaN,NaN,"['typhoon', 'soudelor', 'kill', 'china', 'taiw..."


In [137]:
count_vector = CountVectorizer(ngram_range = (1,2))
Z = count_vector.transform(datatest['text'])

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

In [136]:
#tvect = TfidfVectorizer(min_df=1, max_df=1)
#Z = tvect.transform(datatest)
#datatest['target'] = datatest['transformtarget'].map(lr.predict(Z))
#datatest.head(20)

In [ ]:
y_test = lr.predict(Z)
output = pd.DataFrame()
output['id'] = test['id']
output['target'] = y_test
output.to_csv('submission.csv', index=False)

In [ ]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
>>> X_new_counts = count_vect.transform(docs_new)
>>> X_new_tfidf = tfidf_transformer.transform(X_new_counts)

>>> predicted = clf.predict(X_new_tfidf)

>>> for doc, category in zip(docs_new, predicted):
...     print('%r => %s' % (doc, twenty_train.target_names[category]))
...
'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics
